# Intelligent Decisions for Logistics: An Introduction to Bayesian Optimization
---
**Target Audience:** Medior Business Managers in the Logistics Sector (untrained in Math and Statistics)  
**Duration:** 20 Minutes

## Part 1: Your "Smart Logistics" Goals

By the end of this session, you will be able to:

* **🎯 Identify Opportunities for Smarter Testing:** Spot areas in your operations where traditional 'trial and error' is too slow or expensive.

* **🚀 Understand How to Get Better Results, Faster:** Grasp how Bayesian Optimization can find the best solutions (e.g., for a new routing algorithm or warehouse layout) using fewer, smarter tests.

* **🗣️ Speak the Language of Intelligent Optimisation:** Understand the core ideas behind 'smart predictors' and 'smart decision-makers' to have more informed conversations with your data science teams.

* **⚖️ Recognise the Power of Balanced Strategy:** Understand why a balanced approach of 'checking new things' (exploration) and 'fine-tuning current bests' (exploitation) is crucial for innovation.

## Part 2: Program Outline

#### Introduction (2 minutes)
* **The Challenge in Logistics:** You face complex decisions every day – optimising routes, warehouse layouts, or inventory strategies. Finding the "best" solution often involves a lot of trial-and-error.
* **The Problem with Traditional Approaches:** In logistics, each 'trial' can be incredibly expensive in time, fuel, and labour. You can't just run millions of tests.
* **Introducing Bayesian Optimization (BO):** Imagine an intelligent assistant that learns from every test you run and then suggests the *absolute smartest next test* to get you to the best solution as quickly as possible. That’s BO in a nutshell.

### 1. The Problem: Expensive "Black-Box" Optimisation (4 minutes)

**What is a "Black-Box"?**

Think of a new route planning software. You enter settings (like vehicle capacity or delivery windows), and it gives you an efficiency score. You can't see the complex code inside—it’s a "black box." You only know the inputs and the outputs.

**Why is finding the best setting "Expensive"?**

Evaluating a new logistics strategy involves real-world tests or complex simulations that cost:

* **Time:** A pilot test for a new routing algorithm can take weeks.
* **Money:** Fuel, driver wages, and potential disruptions.
* **Resources:** Vehicles, staff, and computing power.

**The Dilemma:** You want the best result, but testing every possible combination is impossible. Random guesses are inefficient. We need a smarter strategy.

### 2. Bayesian Optimization: The Smart Strategy (7 minutes)

BO uses two components in a cycle: a **"Smart Predictor"** and a **"Smart Decision Maker"**.

#### 2.1. The "Smart Predictor" (Surrogate Model)

This is a statistical model that learns from your test results to "guess" what the black-box function looks like everywhere else. It's like an experienced manager who can predict how a new route might perform based on their knowledge of similar ones.

Crucially, it also tells you **how confident** it is in its prediction.

* **The Solid Line:** The model's "best guess."
* **The Shaded Area:** The uncertainty. A wide area means "we don't know much here." A narrow one means "we're pretty sure."

**👇 Interactive Demo:** Click on the graph below to add test results. See how the "Smart Predictor" updates its belief and reduces uncertainty with each click.

In [1]:
import numpy as np
import plotly.graph_objects as go
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from plotly.subplots import make_subplots

# 1. Define the true "black-box" function (for demonstration purposes)
# In a real scenario, we would not know this function.
def black_box_function(x):
    """A complex function representing logistics efficiency vs. a parameter."""
    return np.sin(0.9 * x) * np.sinc(x * 0.2) * 50 + 25

# 2. Setup for the plot
x_range = np.linspace(0, 20, 200).reshape(-1, 1)
y_true = black_box_function(x_range)

# Initial sample points
X_samples = np.array([[2.0], [18.0]])
y_samples = black_box_function(X_samples)

# Kernel for the Gaussian Process
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))

# Create the figure
fig = go.FigureWidget(make_subplots(rows=1, cols=1))
fig.add_trace(go.Scatter(x=x_range.ravel(), y=y_true.ravel(), mode='lines', 
                         line=dict(color='red', dash='dash'), name='True Efficiency (Unknown)'), row=1, col=1)

# Add initial components to the plot
fig.add_trace(go.Scatter(x=X_samples.ravel(), y=y_samples.ravel(), mode='markers', 
                         marker=dict(color='red', size=12, symbol='x'), name='Test Results'), row=1, col=1)

fig.add_trace(go.Scatter(x=[], y=[], line=dict(color='rgba(0,100,80,0.2)'), showlegend=False), row=1, col=1) # Upper bound
fig.add_trace(go.Scatter(x=[], y=[], line=dict(color='rgba(0,100,80,0.2)'), fill='tonexty', fillcolor='rgba(0,100,80,0.2)', name='Uncertainty'), row=1, col=1) # Lower bound
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', line=dict(color='green', width=3), name='"Smart Predictor" Guess'), row=1, col=1)

def update_gp(X, y):
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, random_state=0)
    gp.fit(X, y)
    y_pred, sigma = gp.predict(x_range, return_std=True)
    return y_pred.ravel(), sigma.ravel()

def update_plot(X, y):
    y_pred, sigma = update_gp(X, y)
    
    # Update GP prediction and uncertainty
    fig.data[2].x = x_range.ravel()
    fig.data[2].y = y_pred + 1.96 * sigma
    fig.data[3].x = x_range.ravel()
    fig.data[3].y = y_pred - 1.96 * sigma
    fig.data[4].x = x_range.ravel()
    fig.data[4].y = y_pred
    
    # Update sample points
    fig.data[1].x = X.ravel()
    fig.data[1].y = y.ravel()
    
    
# Function to handle clicks
def on_click(trace, points, state):
    if points.point_inds:
        # Get click location
        x_click = points.xs[0]
        y_click = black_box_function(np.array([[x_click]]))
        
        # Update global samples
        global X_samples, y_samples
        X_samples = np.vstack([X_samples, [x_click]])
        y_samples = np.vstack([y_samples, y_click])
        
        # Redraw the plot
        update_plot(X_samples, y_samples)

# Attach click handler to the first trace (the invisible background)
fig.data[0].on_click(on_click)

# Initial plot render
update_plot(X_samples, y_samples)

fig.update_layout(title='Interactive Demo: The "Smart Predictor"',
                  xaxis_title='Route Configuration Parameter',
                  yaxis_title='Logistics Efficiency Score',
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.



FigureWidget({
    'data': [{'line': {'color': 'red', 'dash': 'dash'},
              'mode': 'lines',
              'name': 'True Efficiency (Unknown)',
              'type': 'scatter',
              'uid': '066c4920-258d-4a47-ad72-400c2eedd982',
              'x': {'bdata': ('AAAAAAAAAACBhJ9ciLq5P4GEn1yIus' ... '+KzDNAfGCjd0XmM0AAAAAAAAA0QA=='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAOUD8pI5LcYM9QMaKDfBs/E' ... '8jbzlAaTXK4Cs0OUAAAAAAAAA5QA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'marker': {'color': 'red', 'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'name': 'Test Results',
              'type': 'scatter',
              'uid': '49a7985a-8bf3-49ba-aa35-5bd26396b934',
              'x': {'bdata': 'AAAAAAAAAEAAAAAAAAAyQA==', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'v7l7XgTtTkAJFEHrgPw6QA==', 'dtype': 'f8'},
            

#### 2.2. The "Smart Decision Maker" (Acquisition Function)

This component takes the predictor's output (guess + uncertainty) and decides **where you should run your next expensive test.**

It performs a balancing act between:

* **🔵 Exploitation:** "Let's test in an area that the predictor thinks is *already good* to fine-tune our best solution."
* **🟢 Exploration:** "Let's test in an area where the predictor is *very uncertain*, because there might be a hidden, much better solution we don't know about."

The function below is called an **Acquisition Function**. The peak of this function shows the most valuable place to test next—the spot with the best combination of high predicted performance and high uncertainty.

**👇 Interactive Demo:** This chart is linked to the one above. Click on the top chart to add points and watch how the "Smart Decision Maker" changes its recommendation for the next best test.

In [2]:
from scipy.stats import norm

# Let's create a combined, more powerful interactive widget for the whole BO loop

# --- SETUP ---
x_range_bo = np.linspace(0, 20, 200).reshape(-1, 1)
y_true_bo = black_box_function(x_range_bo)

X_samples_bo = np.array([[2.0], [18.0]])
y_samples_bo = black_box_function(X_samples_bo)

kernel_bo = C(1.0, (1e-3, 1e3)) * RBF(5, (1e-2, 1e2))

# --- ACQUISITION FUNCTION ---
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei.ravel()

# --- FIGURE WIDGET ---
fig_bo = go.FigureWidget(make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, 
                                      subplot_titles=("1. The 'Smart Predictor' updates its belief", 
                                                    "2. The 'Smart Decision Maker' suggests the next test")))

# --- PLOT UPDATE LOGIC ---
def update_bo_plot(X, y):
    # Fit GP
    gpr = GaussianProcessRegressor(kernel=kernel_bo, n_restarts_optimizer=10, random_state=0)
    gpr.fit(X, y)
    y_pred, sigma = gpr.predict(x_range_bo, return_std=True)
    
    # Calculate Acquisition
    ei = expected_improvement(x_range_bo, X, y, gpr)
    next_x = x_range_bo[np.argmax(ei)]
    
    with fig_bo.batch_update():
        # Clear previous traces
        fig_bo.data = []

        # --- Top Plot: GP ---
        fig_bo.add_trace(go.Scatter(x=x_range_bo.ravel(), y=y_true_bo.ravel(), mode='lines', line=dict(color='red', dash='dash'), name='True Efficiency (Unknown)'), row=1, col=1)
        fig_bo.add_trace(go.Scatter(x=X.ravel(), y=y.ravel(), mode='markers', marker=dict(color='red', size=12, symbol='x'), name='Test Results'), row=1, col=1)
        fig_bo.add_trace(go.Scatter(x=x_range_bo.ravel(), y=y_pred + 1.96 * sigma, line=dict(color='rgba(0,100,80,0.2)'), showlegend=False), row=1, col=1)
        fig_bo.add_trace(go.Scatter(x=x_range_bo.ravel(), y=y_pred - 1.96 * sigma, line=dict(color='rgba(0,100,80,0.2)'), fill='tonexty', fillcolor='rgba(0,100,80,0.2)', name='Uncertainty'), row=1, col=1)
        fig_bo.add_trace(go.Scatter(x=x_range_bo.ravel(), y=y_pred, mode='lines', line=dict(color='green', width=3), name='Predictor Guess'), row=1, col=1)
        
        # --- Bottom Plot: Acquisition Function ---
        fig_bo.add_trace(go.Scatter(x=x_range_bo.ravel(), y=ei, mode='lines', line=dict(color='blue', width=3), name='Next Test Value Score'), row=2, col=1)
        fig_bo.add_trace(go.Scatter(x=[next_x], y=[np.max(ei)], mode='markers', marker=dict(color='purple', size=14, symbol='star'), name='Recommended Next Test'), row=2, col=1)
        
        # --- Layout ---
        fig_bo.update_layout(height=600, legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
        fig_bo.update_yaxes(title_text="Efficiency Score", row=1, col=1)
        fig_bo.update_yaxes(title_text="Value Score", row=2, col=1)
        fig_bo.update_xaxes(title_text="Route Configuration Parameter", row=2, col=1)

# --- BUTTON INTERACTION ---
from ipywidgets import Button, VBox

button = Button(description="Run Next Smart Test!", button_style='success', icon='lightbulb-o')
reset_button = Button(description="Reset", button_style='warning', icon='refresh')

def on_button_click(b):
    global X_samples_bo, y_samples_bo
    gpr = GaussianProcessRegressor(kernel=kernel_bo, n_restarts_optimizer=10, random_state=0).fit(X_samples_bo, y_samples_bo)
    ei = expected_improvement(x_range_bo, X_samples_bo, y_samples_bo, gpr)
    next_x = x_range_bo[np.argmax(ei)]
    next_y = black_box_function(next_x)
    
    X_samples_bo = np.vstack([X_samples_bo, next_x])
    y_samples_bo = np.vstack([y_samples_bo, next_y])
    
    update_bo_plot(X_samples_bo, y_samples_bo)

def on_reset_click(b):
    global X_samples_bo, y_samples_bo
    X_samples_bo = np.array([[2.0], [18.0]])
    y_samples_bo = black_box_function(X_samples_bo)
    update_bo_plot(X_samples_bo, y_samples_bo)

button.on_click(on_button_click)
reset_button.on_click(on_reset_click)

update_bo_plot(X_samples_bo, y_samples_bo) # Initial plot

VBox([fig_bo, button, reset_button])

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.



IndexError: boolean index did not match indexed array along axis 1; size of axis is 200 but size of corresponding boolean axis is 1

### 3. The Bayesian Optimization Loop: Continuous Improvement (4 minutes)

This is where it all comes together. The process is a simple, powerful loop:

1.  **Start:** Run a few initial tests to get baseline data.
2.  **Predict:** The "Smart Predictor" (top graph) models the efficiency landscape based on all current results.
3.  **Decide:** The "Smart Decision Maker" (bottom graph) identifies the single most valuable place for the next test.
4.  **Test:** You run that one test on your real-world system.
5.  **Learn & Repeat:** Add the new result to your data, and the loop repeats. The model gets smarter with every cycle.

Use the **"Run Next Smart Test!"** button in the demo above to step through this loop. Notice how the algorithm first explores uncertain areas and then begins to exploit the most promising region to find the optimal solution much faster than random guessing.

### 4. Bayesian Optimization in Logistics: Real-World Impact (2 minutes)
BO can provide a significant competitive advantage in logistics by driving efficiency and cost savings. Key applications include:

* **🤖 Optimising ML Models:** Finding the best settings (hyperparameters) for demand forecasting or delivery time estimation models.

* **🚚 Route & Network Optimisation:** Fine-tuning the parameters of a vehicle routing algorithm to minimise fuel or reduce delivery times.

* **🏭 Warehouse Robotics:** Developing optimal policies for how robots move and sort items in an automated warehouse by using expensive simulations more effectively.

* **⛓️ Supply Chain Design:** Finding optimal locations for new distribution centres by balancing costs, lead times, and service levels.

* **📦 Inventory Management:** Optimising inventory control policies under uncertain demand.

### Conclusion & Call to Action (1 minute)

* **Key Takeaway:** Bayesian Optimization is an intuitive method for solving expensive, 'black-box' problems. It finds the best solutions faster by combining smart predictions with intelligent decision-making.

* **Your Advantage:** Reducing the number of expensive trials provides a significant competitive advantage, driving efficiency, cost savings, and innovation.

* **Next Steps:** Think about where your operations face expensive trial-and-error. **Talk to your data science or R&D teams** about exploring Bayesian Optimization for these challenges.